ipython kernel install --name "cac2020" --user


In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [10]:
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dense, Dropout, Flatten
from tensorflow_core.python.keras.callbacks import ModelCheckpoint

In [11]:
# from keras_tdqm import TQDMNotebookCallback
import json

In [12]:
train_image_path = "C:\\Users\\neelj\\Documents\\Python\\fruits-360" \
                   "\\simplified\\Training-modified"
test_image_path = "C:\\Users\\neelj\\Documents\\Python\\fruits-360" \
                  "\\simplified\\Test-modified"

batch_size = 128
num_classes = 20
num_epochs = 20
img_rows, img_cols = 100, 100
optimizer = Adam()

In [13]:
train_data_gen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
)
validation_data_gen = ImageDataGenerator(rescale=1./255,
                                         validation_split=0.2,)

train_data_gen = train_data_gen.flow_from_directory(
    train_image_path,
    # save_to_dir="C:\\Users\\neelj\\Documents\\Python\\fruits-360" \
    # "\\modified-sets\\training-processed",
    # save_format="jpeg",
    target_size=(img_rows, img_cols),
    batch_size=batch_size,
    class_mode="categorical",
    subset="training",
)

validation_data_gen = validation_data_gen.flow_from_directory(
    train_image_path,
    target_size=(img_rows, img_cols),
    batch_size=batch_size,
    class_mode="categorical",
    subset="validation",
)

Found 7961 images belonging to 20 classes.
Found 1983 images belonging to 20 classes.


In [17]:
model = Sequential()
model.add(Conv2D(16, kernel_size=(2, 2),
                 activation="relu", input_shape=(img_cols, img_rows, 3),
                padding='same'))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=2))

model.add(Conv2D(64, kernel_size=(2, 2), activation="relu", padding='same'))
model.add(MaxPooling2D(pool_size=2))

model.add(Conv2D(128, kernel_size=(2, 2), activation="relu", padding='same'))
model.add(MaxPooling2D(pool_size=2))

model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(150, activation="relu"))
model.add(Dropout(0.4))
model.add(Dense(num_classes, activation="sigmoid"))

model.compile(loss="binary_crossentropy",
              optimizer=optimizer,
              metrics=["accuracy"],)

model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 100, 100, 16)      208       
_________________________________________________________________
activation_3 (Activation)    (None, 100, 100, 16)      0         
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 50, 50, 16)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 50, 50, 64)        4160      
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 25, 25, 64)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 25, 25, 128)       32896     
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 12, 12, 128)      

In [18]:
"""
CNN_model = model.fit(
    train_data_gen,
    epochs=num_epochs,
    shuffle=True,
    verbose=2,
    validation_data=validation_data_gen,
)
"""

CNN_model = model.fit_generator(
    train_data_gen,
    steps_per_epoch=train_data_gen.samples // batch_size,
    validation_data=validation_data_gen,
    validation_steps=validation_data_gen.samples // batch_size,
    epochs=num_epochs,
    verbose=1,
)

Epoch 1/20
62/62 [==============================] - 58s 936ms/step - loss: 0.1471 - accuracy: 0.9511 - val_loss: 0.0293 - val_accuracy: 0.9896
Epoch 2/20
54/62 [=========================>....] - ETA: 7s - loss: 0.0172 - accuracy: 0.9940

KeyboardInterrupt: 

In [16]:
test_data_gen = ImageDataGenerator(rescale=1./255)

test_data_gen = test_data_gen.flow_from_directory(
    train_image_path,
    target_size=(img_rows, img_cols),
    batch_size=batch_size,
    class_mode="categorical",
)

model.evaluate_generator(test_data_gen,
                        verbose=1,)

Found 9944 images belonging to 20 classes.
78/78 [==============================] - 17s 221ms/step


[nan, 0.049477070569992065]

In [9]:
idx_to_cls = {v: k for k, v in train_data_gen.class_indices.items()}
print(idx_to_cls)

{0: 'Apple Red Yellow', 1: 'Banana', 2: 'Blueberry', 3: 'Cauliflower', 4: 'Corn', 5: 'Eggplant', 6: 'Grape White', 7: 'Lemon', 8: 'Lychee', 9: 'Mandarine', 10: 'Onion White', 11: 'Pear', 12: 'Pepper Green', 13: 'Pepper Red', 14: 'Pomegranate', 15: 'Potato Red', 16: 'Raspberry', 17: 'Strawberry', 18: 'Tomato', 19: 'Watermelon'}


In [10]:
with open("s_class-indicies.json", "w") as f:
    json.dump(idx_to_cls, f, indent=4)

In [20]:

predict_generator = ImageDataGenerator(rescale=1/255.)

predict_generator = predict_generator.flow_from_directory("C:\\Users\\neelj\\PycharmProjects\\cac2020\\classifier",
                              classes=["test-images"],
                              class_mode=None,
                              shuffle=False,
                              target_size=(img_rows, img_cols),
                              )

predictions = model.predict_generator(predict_generator)

print(predictions)
preds_cls_idx = predictions.argmax(axis=1)
print(preds_cls_idx)

Found 1 images belonging to 1 classes.
[[0.         0.         0.         0.97308064 0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.        ]]
[3]


In [41]:
model_json = model.to_json()
with open("model.json", "w") as f:
    f.write(model_json)
model.save_weights("model.h5")

ALTERNATIVE MODEL Design that may work idk